In [28]:
import re
import json
import glob
import pickle
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy

In [2]:
df = pd.read_csv("Clean_data/Kickstarter.csv")

In [3]:
urls = df.urls
ids = df.id

urls_ = list(map(lambda url:json.loads(url)["web"]["project"],urls))
urls = list(map(lambda url:url.replace("?ref=discovery_category_newest","/description"),urls_))

In [7]:
errors = np.zeros(len(df))
# storys = [""]*len(urls)

path = "Webdriver/chromedriver.exe"
options = webdriver.ChromeOptions()
options.add_argument('--lang=en')
# 0

In [75]:
driver = webdriver.Chrome(executable_path = path,chrome_options=options)
for i in tqdm(range(num,num + 100)):
    try:
        driver.get(urls[i])
        story_raw = WebDriverWait(driver,10).until(
        EC.presence_of_element_located((By.XPATH,"//div[contains(@class, 'rte__content')]"))
        )
        storys[i] = story_raw.text
    except:
        errors[i] = 1
    finally:
        time.sleep(np.random.randint(200)/100)
driver.quit()

<ipython-input-75-76b8a37add20>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = path,chrome_options=options)
100%|██████████| 100/100 [08:04<00:00,  4.85s/it]


In [78]:
num = 1000
sum(errors)

15.0

In [80]:
driver = webdriver.Chrome(executable_path = path,chrome_options=options)
for i in range(1000):
    if(errors[i] > 0):
        try:
            driver.get(urls[i])
            story_raw = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.XPATH,"//div[contains(@class, 'rte__content')]"))
            )
            storys[i] = story_raw.text
            errors[i] = 0
        except:
            pass
        finally:
            time.sleep(np.random.randint(200)/100)
driver.quit()

<ipython-input-80-3bff04e2f4d2>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = path,chrome_options=options)


In [83]:
sum(errors)

0.0

In [84]:
df_export = pd.DataFrame({"id":df["id"],
                         "story":storys}, index = None)
df_export.to_csv("Output/Story.csv")